What I need to do here to make sure that there is no confusion about the positions of opposite homozygotes is to call SNPs using the joint calling with the 2K matrix.

In [1]:
source("../../BrusselSprouts/scripts/functions.R")


scripts_dir="/home/jnrunge/data/trd/mapped_reads/scripts/"

initial_timedate=Sys.time()

# what needs to be done is individually, and parallely genotype all crosses, then merge them into a DB, and once call the joint genotyping (once per batch of sequencing, so twice in total)

bam_files=list.files(path="~/data/trd/mapped_reads",pattern="JNRC[0-9]*.bam$",full.names = TRUE)
bam_files=naturalsort(bam_files)
crosses=unlist(lapply(basename(bam_files), getFirst_v2, split="."))

for(cross in crosses){
    print(cross)

    if(file.exists(paste0("/home/jnrunge/data/trd/mapped_reads/",cross,".g.vcf.gz"))){
        print("GVCF exists")
        next
    }
    file.create(running_file<-paste0("~/data/TRD/",cross,".runningGATK"))
    cmds=paste0("cd /home/jnrunge/data/trd/mapped_reads/ && gatk AddOrReplaceReadGroups -I ",cross,".bam -O ",cross,".ReadGroups.bam --RGID ",cross," --RGLB ",cross," --RGPL ILLUMINA --RGPU ",cross," --RGSM ",cross,"")
    cmds=c(cmds,paste0("mv -f ",cross,".ReadGroups.bam ",cross,".bam"))
    cmds=c(cmds,paste0("samtools index ",cross,".bam"))
    cmds=c(cmds,paste0("gatk HaplotypeCaller -R ../../TRD/R64_nucl.fasta -I ",cross,".bam -O ",cross,".g.vcf.gz --emit-ref-confidence GVCF && rm -f ","/home/jnrunge/data/trd/mapped_reads/",cross,".g.vcf.gz"))

    sbatch_list=execute_complex_sbatch(cmds, jobname = jobname<-"GATK", scripts_dir = scripts_dir, uniqueRunID = cross, cores="1", mem="32gb", time="short", env="bwaetc", initial_timedate = initial_timedate, jobs_simul = 10, jobs_total = 10)


}

if(exists("sbatch_list")){
    print("Batching up...")
    start_sbatch_list(sbatch_list, 10, jobname, initial_timedate)
    }

while(length(list.files(path = "~/data/TRD", pattern = ".runningGATK$"))>0){
    Sys.sleep(60)
}


Attaching package: 'dplyr'


The following objects are masked from 'package:data.table':

    between, first, last


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




[1] "YJNRC2"
[1] "GVCF exists"
[1] "YJNRC17"
[1] "GVCF exists"
[1] "YJNRC18"
[1] "GVCF exists"


In [2]:
library(readxl)

crosses=list.files("~/data/trd/mapped_reads",pattern = "g.vcf.gz$",full.names = TRUE)

my_crosses=read_xlsx("~/data/trd/Crosses.xlsx", sheet=2)[,c("Short name 1", "Short name 2", "Status", "Colonies in pool")]
chris_crosses=data.frame(sample1=c("ACP","BAP","CCD","ATE","ACK","AKE","BAH","ANG"),
                        sample2=c("BFP","CMP","CPG","YCR","ACV","BAH","CGD","CEI"))
all_used_samples=unique(c(my_crosses$`Short name 1`, my_crosses$`Short name 2`, chris_crosses$sample1, chris_crosses$sample2))
all_used_samples[all_used_samples=="YCR"]=paste0("SACE_",all_used_samples[all_used_samples=="YCR"])
summary(file.exists(paste0("/home/jnrunge/data/trd/GVCF_2489Strains/",all_used_samples,".g.vcf.gz")))

all_used_samples[!file.exists(paste0("/home/jnrunge/data/trd/GVCF_2489Strains/",all_used_samples,".g.vcf.gz"))]


samples=c(crosses,paste0("/home/jnrunge/data/trd/GVCF_2489Strains/",all_used_samples,".g.vcf.gz"))
samples=data.frame(id=unlist(lapply(basename(samples), getFirst_v2, split=".")),file=samples)
fwrite(samples, "~/data/trd/mapped_reads/all_samples.tsv", col.names = FALSE, sep="\t")
head(samples)

New names:
* `Is in Stock` -> `Is in Stock...4`
* `Is in Stock` -> `Is in Stock...6`


   Mode    TRUE 
logical      57 

character(0)

,id,file
,<chr>,<chr>
1,YJNRC17,/home/jnrunge/data/trd/mapped_reads/YJNRC17.g.vcf.gz
2,YJNRC18,/home/jnrunge/data/trd/mapped_reads/YJNRC18.g.vcf.gz
3,YJNRC2,/home/jnrunge/data/trd/mapped_reads/YJNRC2.g.vcf.gz
4,BAK,/home/jnrunge/data/trd/GVCF_2489Strains/BAK.g.vcf.gz
5,BAN,/home/jnrunge/data/trd/GVCF_2489Strains/BAN.g.vcf.gz
6,AKH,/home/jnrunge/data/trd/GVCF_2489Strains/AKH.g.vcf.gz


In [3]:
# make BED file (all positions because GATK is retarded)
fai=fread("~/data/TRD/R64_nucl.fasta.fai")
bed=data.frame(chr=fai$V1, from=1, to=fai$V2)
head(bed)
fwrite(bed, "~/data/TRD/R64_nucl.fasta.fai.bed", col.names = FALSE, sep="\t")

,chr,from,to
,<chr>,<dbl>,<int>
1,chromosome1,1,230218
2,chromosome2,1,813184
3,chromosome3,1,316620
4,chromosome4,1,1531933
5,chromosome5,1,576874
6,chromosome6,1,270161


In [ ]:
if(file.mtime("~/data/trd/mapped_reads/TRD.vcf.gz")<max(file.mtime(samples$file))){
    # if new samples can be added to the big vcf, then merge again
    
    file.create(running_file<-paste0("~/data/TRD/runningGATKmerge"))
    cmd="cd ~/data/trd/mapped_reads && rm -rf TRD_DB"
    cmd=paste0(cmd, " && gatk GenomicsDBImport --batch-size 200 --genomicsdb-workspace-path TRD_DB --sample-name-map ~/data/trd/mapped_reads/all_samples.tsv -L ~/data/TRD/R64_nucl.fasta.fai.bed")
    cmd=paste0(cmd, " && gatk GenotypeGVCFs -R ../../TRD/R64_nucl.fasta -V gendb://TRD_DB -O TRD.vcf.gz")
    cmd=paste0(cmd, " && bcftools query -l TRD.vcf.gz > TRD.vcf.gz.samples && rm -f ~/data/TRD/runningGATKmerge")
    execute_cmd_sbatch(cmd, mem="32G", cpu="1", time="long", env="bwaetc", jobname="GATK_merge")
    
    while(file.exists(running_file)){
    Sys.sleep(60)
}
    
}


In [29]:
Crosses=readxl::read_xlsx("~/data/trd/Crosses.xlsx", sheet=2)
head(Crosses<-Crosses[,c("Cross ID","Short name 1","Short name 2")])
tail(Crosses)

New names:
* `Is in Stock` -> `Is in Stock...4`
* `Is in Stock` -> `Is in Stock...6`


Cross ID,Short name 1,Short name 2
<chr>,<chr>,<chr>
YJNRC1,BAK,BET
YJNRC2,BAN,BTI
YJNRC3,AKH,BQC
YJNRC4,ABS,BEF
YJNRC5,BAK,BMK
YJNRC6,BAN,CRB


Cross ID,Short name 1,Short name 2
<chr>,<chr>,<chr>
YJNRC29,AAR,CPG
YJNRC30,ABG,CPG
YJNRC31,AKI,CPG
YJNRC32,ATE,CPG
YJNRC33,AAR,AKH
YJNRC34,AKH,AKQ


In [32]:
crosses=unlist(lapply(basename(bam_files), getFirst_v2, split="."))

for(cross in crosses){
    run=FALSE
    print(cross)
    gvcf<-paste0("/home/jnrunge/data/trd/mapped_reads/",cross,".g.vcf.gz")
    if(file.exists(ohfile<-paste0("~/data/trd/mapped_reads/",cross,".hetLoci.gz"))){
        if(file.mtime(gvcf)>file.mtime(ohfile)){
            run=TRUE
            
        }
    }else{
        run=TRUE
    }
    if(run == TRUE){
            file.create(running_file<-paste0("~/data/TRD/Oppo-Homo-Pos/",cross,".runningOHLOCI"))
            cmd=paste0("cd /home/jnrunge/data/trd/mapped_reads/ && sh ~/TRD/01_Mapping/02_OHLoci2.sh ",cross)
            cmd=paste0(cmd," && rm -f ",running_file)
            execute_cmd_sbatch(cmd, mem="8G", cpu="1", time="short", env="bwaetc", jobname = "OHLoci")
            print(cmd)
    }
    }
while(length(list.files(path = "~/data/TRD/Oppo-Homo-Pos", pattern = ".runningOHLOCI$"))>0){
    Sys.sleep(60)
}

[1] "YJNRC2"
[1] "sbatch -c 1 --mem=8G --job-name=OHLoci -p fast -t 11:59:00 --wrap '. ~/activate.sh bwaetc; cd /home/jnrunge/data/trd/mapped_reads/ && sh ~/TRD/01_Mapping/02_OHLoci2.sh YJNRC2 && rm -f ~/data/TRD/Oppo-Homo-Pos/YJNRC2.runningOHLOCI'"
[1] "Submitted batch job 481327"
[1] "cd /home/jnrunge/data/trd/mapped_reads/ && sh ~/TRD/01_Mapping/02_OHLoci2.sh YJNRC2 && rm -f ~/data/TRD/Oppo-Homo-Pos/YJNRC2.runningOHLOCI"
[1] "YJNRC17"
[1] "sbatch -c 1 --mem=8G --job-name=OHLoci -p fast -t 11:59:00 --wrap '. ~/activate.sh bwaetc; cd /home/jnrunge/data/trd/mapped_reads/ && sh ~/TRD/01_Mapping/02_OHLoci2.sh YJNRC17 && rm -f ~/data/TRD/Oppo-Homo-Pos/YJNRC17.runningOHLOCI'"
[1] "Submitted batch job 481328"
[1] "cd /home/jnrunge/data/trd/mapped_reads/ && sh ~/TRD/01_Mapping/02_OHLoci2.sh YJNRC17 && rm -f ~/data/TRD/Oppo-Homo-Pos/YJNRC17.runningOHLOCI"
[1] "YJNRC18"
[1] "sbatch -c 1 --mem=8G --job-name=OHLoci -p fast -t 11:59:00 --wrap '. ~/activate.sh bwaetc; cd /home/jnrunge/data/trd/map

In [33]:
for(s in samples$id){
    if(grepl("YJNRC",s,fixed = TRUE)){
        next
    }
    run=FALSE
    print(s)
    if(file.exists(ohfile<-paste0("~/data/trd/mapped_reads/",s,".homLoci.gz"))){
        if(file.mtime(gvcf)>file.mtime(ohfile)){
            run=TRUE
            
        }
    }else{
        run=TRUE
    }
    if(run == TRUE){
            file.create(running_file<-paste0("~/data/TRD/Oppo-Homo-Pos/",s,".runningOHLOCI"))
            cmd=paste0("cd /home/jnrunge/data/trd/mapped_reads/ && sh ~/TRD/01_Mapping/02_OHLoci3.sh ",s)
            cmd=paste0(cmd," && rm -f ",running_file)
            execute_cmd_sbatch(cmd, mem="8G", cpu="1", time="short", env="bwaetc", jobname = "OHLoci")
            print(cmd)
        }
    }
while(length(list.files(path = "~/data/TRD/Oppo-Homo-Pos", pattern = ".runningOHLOCI$"))>0){
    Sys.sleep(60)
}

[1] "BAK"
[1] "sbatch -c 1 --mem=8G --job-name=OHLoci -p fast -t 11:59:00 --wrap '. ~/activate.sh bwaetc; cd /home/jnrunge/data/trd/mapped_reads/ && sh ~/TRD/01_Mapping/02_OHLoci3.sh BAK && rm -f ~/data/TRD/Oppo-Homo-Pos/BAK.runningOHLOCI'"
[1] "Submitted batch job 481332"
[1] "cd /home/jnrunge/data/trd/mapped_reads/ && sh ~/TRD/01_Mapping/02_OHLoci3.sh BAK && rm -f ~/data/TRD/Oppo-Homo-Pos/BAK.runningOHLOCI"
[1] "BAN"
[1] "sbatch -c 1 --mem=8G --job-name=OHLoci -p fast -t 11:59:00 --wrap '. ~/activate.sh bwaetc; cd /home/jnrunge/data/trd/mapped_reads/ && sh ~/TRD/01_Mapping/02_OHLoci3.sh BAN && rm -f ~/data/TRD/Oppo-Homo-Pos/BAN.runningOHLOCI'"
[1] "Submitted batch job 481333"
[1] "cd /home/jnrunge/data/trd/mapped_reads/ && sh ~/TRD/01_Mapping/02_OHLoci3.sh BAN && rm -f ~/data/TRD/Oppo-Homo-Pos/BAN.runningOHLOCI"
[1] "AKH"
[1] "sbatch -c 1 --mem=8G --job-name=OHLoci -p fast -t 11:59:00 --wrap '. ~/activate.sh bwaetc; cd /home/jnrunge/data/trd/mapped_reads/ && sh ~/TRD/01_Mapping/02_OH